In [25]:
from toolz import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
%matplotlib widget


import pathlib
from lenses import lens

from collections import Counter

import nltk
import pymorphy2
import fasttext.util

In [3]:
nltk.download('popular')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('tagsets')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to

True

In [4]:
lmap = compose(list, map)
ltake = compose(list, take)

In [5]:
def map_df_ingredients_element(func, df):
    df = df.copy()
    df['ingredients'] = df['ingredients'].map(lambda v: lmap(func, v))
    return df

In [6]:
data_path = pathlib.Path('data')

## Загружаем датафрейм с данными

In [7]:
df = pd.read_json(data_path.joinpath('ready_dataframe.json'))
df = map_df_ingredients_element(compose(str.lower, str.strip), df)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"[бананы, лимонный сок, молотая корица, ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"[сахар, кокосовое молоко, корень имбиря, анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"[клубника, манго, сахар, тертая цедра лайма, т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"[манго, темный ром, сок лайма, сахар, тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"[клубника, сахарная пудра, сок лайма, текила, ..."
...,...,...,...,...,...
41433,43380,Постный борщ с фасолью,Супы,Украинская кухня,"[белая фасоль, овощной бульон, свекла, картофе..."
41434,80446,Суп энгамат,Супы,Шведская кухня,"[цветная капуста, морковь, картофель, лук-поре..."
41435,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"[горох, репчатый лук, гвоздика, свиная рулька,..."
41436,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"[оливковое масло, репчатый лук, овощной бульон..."


## Работа с ингридиентами

Удобнее работать с листом листов:

In [8]:
ingredients = list(df['ingredients'])
# TODO !!!!!!!!! Временно берем только первые несколько записей 
ingredients = ingredients[:20]
ingredients[:3]

[['бананы', 'лимонный сок', 'молотая корица', 'ванильное мороженое'],
 ['сахар', 'кокосовое молоко', 'корень имбиря', 'ананас', 'сок лайма'],
 ['клубника',
  'манго',
  'сахар',
  'тертая цедра лайма',
  'текила',
  'сок лайма',
  'апельсиновый ликер']]

### Самые часто встречающиеся слова в ингридиентах

In [9]:
ingredients_counter = Counter()
for lst in ingredients:
    ingredients_counter.update(lst)
# first_ingresients, _ = zip(*ingredients_counter.most_common(10))
# first_ingresients = list(first_ingresients)
# first_ingresients
ingredients_counter.most_common(10)

[('сахар', 15),
 ('пшеничная мука', 14),
 ('куриное яйцо', 11),
 ('сливочное масло', 9),
 ('соль', 7),
 ('сахарная пудра', 5),
 ('сок лайма', 4),
 ('сметана', 3),
 ('сода', 3),
 ('корица', 3)]

In [10]:
it = lmap(compose(tuple, nltk.word_tokenize), ingredients_counter)
# it

### Токенизация

In [11]:
ingredients_tokenized = lens.Each().Each().modify(nltk.word_tokenize)(ingredients)
ingredients_tokenized[:4]

[[['бананы'],
  ['лимонный', 'сок'],
  ['молотая', 'корица'],
  ['ванильное', 'мороженое']],
 [['сахар'],
  ['кокосовое', 'молоко'],
  ['корень', 'имбиря'],
  ['ананас'],
  ['сок', 'лайма']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['текила'],
  ['сок', 'лайма'],
  ['апельсиновый', 'ликер']],
 [['манго'],
  ['темный', 'ром'],
  ['сок', 'лайма'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['ананас']]]

### Стоп-слова

In [12]:
stopwords = nltk.corpus.stopwords.words('russian')

In [13]:
ingredients_with_stopwords = (lens
    .Each().Each().modify(
        lambda tokenized: list(filter(lambda word: word not in stopwords, tokenized))
    )(ingredients_tokenized)
)
ingredients_with_stopwords[:4]

[[['бананы'],
  ['лимонный', 'сок'],
  ['молотая', 'корица'],
  ['ванильное', 'мороженое']],
 [['сахар'],
  ['кокосовое', 'молоко'],
  ['корень', 'имбиря'],
  ['ананас'],
  ['сок', 'лайма']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['текила'],
  ['сок', 'лайма'],
  ['апельсиновый', 'ликер']],
 [['манго'],
  ['темный', 'ром'],
  ['сок', 'лайма'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['ананас']]]

### Лемматизация

In [14]:
morph = pymorphy2.MorphAnalyzer()

ingredients_lemmatized = lens.Each().Each().Each().modify(
    lambda word: morph.parse(word)[0].normal_form
)(ingredients_with_stopwords)
ingredients_lemmatized[:4]

[[['банан'],
  ['лимонный', 'сок'],
  ['молотый', 'корица'],
  ['ванильный', 'мороженое']],
 [['сахар'],
  ['кокосовый', 'молоко'],
  ['корень', 'имбирь'],
  ['ананас'],
  ['сок', 'лайм']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тёртый', 'цедра', 'лайм'],
  ['текила'],
  ['сок', 'лайм'],
  ['апельсиновый', 'ликёр']],
 [['манго'],
  ['тёмный', 'ром'],
  ['сок', 'лайм'],
  ['сахар'],
  ['тёртый', 'цедра', 'лайм'],
  ['ананас']]]

In [21]:
words = set()
words_count = {}
for i in ingredients_lemmatized:
    for j in i:
        for k in j:
            if k not in words_count.keys(): words_count[k] = 0
            words.add(k)
            words_count[k] += 1
words = list(words)
words.sort()
words[:10]

['%',
 '-ный',
 '20',
 '30',
 '70',
 'абрикосовый',
 'ананас',
 'апельсиновый',
 'банан',
 'белок']

In [26]:
list(words_count.items())[:10]

[('банан', 1),
 ('лимонный', 2),
 ('сок', 7),
 ('молотый', 4),
 ('корица', 5),
 ('ванильный', 5),
 ('мороженое', 1),
 ('сахар', 19),
 ('кокосовый', 1),
 ('молоко', 4)]

In [33]:
#ОЧЕНЬ ДОЛГО
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [34]:
ft = fasttext.load_model('cc.ru.300.bin')
ft.get_dimension()

300

### Получаем вектора для слов 

In [37]:
embeddings_dict = {}
for word in words:
    embeddings_dict[word] = ft.get_word_vector(word)

In [38]:
embeddings_dict

{'%': array([ 0.02615482, -0.05681516, -0.06775013, -0.07192415,  0.03154404,
         0.02907906,  0.2237188 ,  0.18063165,  0.06406087, -0.17652863,
         0.04230109,  0.09412638, -0.46623725,  0.27319682, -0.05929491,
        -0.2070255 , -0.0641679 , -0.12222388,  0.00805531, -0.2547647 ,
         0.07645922,  0.23964563, -0.18577014, -0.27006543, -0.00286642,
         0.08702929,  0.00585793, -0.01460374,  0.04729532,  0.16708533,
        -0.11847235, -0.00449606,  0.11246164,  0.0226811 , -0.41592357,
         0.20552926, -0.4306189 , -0.8096553 ,  0.3004144 ,  0.07227463,
         0.14625055, -0.11910628,  0.04544058, -0.30057135,  0.26404452,
         0.1568386 ,  0.11297377, -0.20403947, -0.02295551, -0.4384995 ,
         0.10422523, -0.12441339,  0.15647481, -0.11375122,  0.23124146,
         0.27602944, -0.21844453, -0.24790128, -0.35540575,  0.00371256,
        -0.21097037, -0.1135601 ,  0.29205018, -0.04209768, -0.09261861,
         0.16079773, -0.03928676,  0.08948038,

In [40]:
word_num = [(word, i) for i, word in enumerate(words)]
word_num[:10]

[('%', 0),
 ('-ный', 1),
 ('20', 2),
 ('30', 3),
 ('70', 4),
 ('абрикосовый', 5),
 ('ананас', 6),
 ('апельсиновый', 7),
 ('банан', 8),
 ('белок', 9)]

## Прочее:

In [20]:
it = df.groupby('cuisine').count()
it.sort_values('title', ascending=False).head(30)

,id,title,course,ingredients
cuisine,,,,
Европейская кухня,11699,11699,11699,11699
Русская кухня,7111,7111,7111,7111
Итальянская кухня,4943,4943,4943,4943
Французская кухня,3055,3055,3055,3055
Американская кухня,2192,2192,2192,2192
Авторская кухня,1822,1822,1822,1822
Китайская кухня,840,840,840,840
Греческая кухня,640,640,640,640
Мексиканская кухня,583,583,583,583
